<a href="https://colab.research.google.com/github/vlastimiltetour/safari_news/blob/main/safari_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium pandas webdriver-manager
import smtplib

ERROR: Could not find a version that satisfies the requirement chromium-browser (from versions: none)
ERROR: No matching distribution found for chromium-browser


In [ ]:
!apt-get update
!apt-get install -y wget gnupg
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (105 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1

In [ ]:
!google-chrome --version

Google Chrome 144.0.7559.59 


In [ ]:
import tempfile
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import time
import re
import psycopg2
import logging
from datetime import datetime

In [ ]:
class WebScraper():

  def scraper_driver(self, url):
    #self.user_data_dir = None
    self.logger = logging.getLogger(__name__)
    # Setup headless Chrome
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox') # Essential for running Chrome in a containerized environment like Colab
    options.add_argument('--disable-dev-shm-usage') # Overcomes limited resource problems
    options.binary_location = "/usr/bin/google-chrome"

    service = Service(ChromeDriverManager().install())

    self.driver = webdriver.Chrome(service=service, options=options)

    #Initialize the Webdriver
    self.wait = WebDriverWait(self.driver, 2)
    self.driver.get(url)

  def scrape(self):
    news_list = []
    self.driver.implicitly_wait(0.5)
    all_news = self.driver.find_elements(by=By.CSS_SELECTOR, value="article.item")
    for news in all_news:
      news = news.text
      news_list.append(news)

    news_clean = [i.replace("\n", " ") for i in news_list]
    html_content = news_clean
    self.driver.quit()
    return html_content

In [ ]:
# Email
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

class EmailSender():

  def send_email(self, subject, sender, recipients, password, html_content):
    # Create email message container
    msg = MIMEMultipart("alternative")
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    # Attach HTML content
    html_part = MIMEText(html_content, "html")
    msg.attach(html_part)

    # Send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender, password)
        smtp_server.sendmail(sender, recipients, msg.as_string())
    print("Message sent!")

In [ ]:
def main():
  # Email configuration
  subject = "Novinky ze Safari"
  sender = "v.tetour@gmail.com"
  recipients = ["vlkscraper@outlook.com", "v.tetour@gmail.com"]
  password = "hyhe qdhx ntsu cvqs"

  webscraper = WebScraper()
  webscraper.scraper_driver("https://safaripark.cz/")
  html_content = webscraper.scrape()

  email_sender = EmailSender()
  email_sender.send_email(subject, sender, recipients, password, html_content=html_content)

if __name__ == "__main__":
  main()

['15.1.2026 Hledáme posilu týmu Zooshopu', '13.1.2026 Safari Park Dvůr Králové hledá chovatele/ku ptáků', '5.1.2026 První mláďě roku 2026!']
